In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mrtontrnok/5-vehichles-for-multicategory-classification")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/5-vehichles-for-multicategory-classification


In [3]:
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.applications import ResNet50

In [4]:
train_path='/kaggle/input/5-vehichles-for-multicategory-classification/dataset/train'
val_path='/kaggle/input/5-vehichles-for-multicategory-classification/dataset/validation'

IMG_SIZE=224
BATCH_SIZE=32
EPOCHS=10

In [5]:
train_ds=tf.keras.utils.image_dataset_from_directory(
    train_path,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

val_ds=tf.keras.utils.image_dataset_from_directory(
    val_path,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

Found 5418 files belonging to 5 classes.


I0000 00:00:1768559723.156025      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Found 709 files belonging to 5 classes.


one hot encoding

In [6]:
num_class=train_ds.cardinality().numpy()

In [9]:
data_augmentaion=tf.keras.Sequential([
    layers.RandomFlip('horizonatal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

In [11]:
from tensorflow.keras.applications.resnet50 import preprocess_input

In [12]:
def preprocess_for_resnet(image,label):
    image=tf.cast(image,tf.float32)
    image=preprocess_input(image)
    return image, label

In [13]:
train_ds=train_ds.map(preprocess_for_resnet)
val_ds=val_ds.map(preprocess_for_resnet)

In [15]:
base_model=ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [16]:
base_model.trainable=False

In [23]:
inputs=tf.keras.Input(shape=(224,224,3))
x=data_augmentaion(inputs)
x=preprocess_input(x)

x=base_model(x, training=False)
x=layers.GlobalAveragePooling2D()(x)
x=layers.Dense(128,activation='relu')(x)
x=layers.Dropout(0.5)(x)

outputs=layers.Dense(
    train_ds.element_spec[1].shape[-1],
    activation='softmax'
)(x)

model=tf.keras.Model(inputs, outputs)

In [24]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [25]:
history=model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

Epoch 1/10


I0000 00:00:1768561749.236675     405 cuda_dnn.cc:529] Loaded cuDNN version 91002


170/170 ━━━━━━━━━━━━━━━━━━━━ 32s 123ms/step - accuracy: 0.6177 - loss: 1.0241 - val_accuracy: 0.8632 - val_loss: 0.3520
Epoch 2/10
170/170 ━━━━━━━━━━━━━━━━━━━━ 17s 99ms/step - accuracy: 0.7941 - loss: 0.5510 - val_accuracy: 0.8745 - val_loss: 0.3200
Epoch 3/10
170/170 ━━━━━━━━━━━━━━━━━━━━ 17s 99ms/step - accuracy: 0.8182 - loss: 0.5021 - val_accuracy: 0.8731 - val_loss: 0.3107
Epoch 4/10
170/170 ━━━━━━━━━━━━━━━━━━━━ 17s 98ms/step - accuracy: 0.8193 - loss: 0.4723 - val_accuracy: 0.8872 - val_loss: 0.2986
Epoch 5/10
170/170 ━━━━━━━━━━━━━━━━━━━━ 17s 97ms/step - accuracy: 0.8340 - loss: 0.4550 - val_accuracy: 0.8942 - val_loss: 0.3083
Epoch 6/10
170/170 ━━━━━━━━━━━━━━━━━━━━ 17s 97ms/step - accuracy: 0.8469 - loss: 0.4154 - val_accuracy: 0.8829 - val_loss: 0.3077
Epoch 7/10
170/170 ━━━━━━━━━━━━━━━━━━━━ 16s 96ms/step - accuracy: 0.8379 - loss: 0.4415 - val_accuracy: 0.8801 - val_loss: 0.3317
Epoch 8/10
170/170 ━━━━━━━━━━━━━━━━━━━━ 17s 96ms/step - accuracy: 0.8410 - loss: 0.4130 - val_accura

In [26]:
base_model.trainable=True
for layer in base_model.layers[:-30]:
    layer.trainable=False

In [27]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [29]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

Epoch 1/10
170/170 ━━━━━━━━━━━━━━━━━━━━ 22s 126ms/step - accuracy: 0.9005 - loss: 0.2635 - val_accuracy: 0.9069 - val_loss: 0.2857
Epoch 2/10
170/170 ━━━━━━━━━━━━━━━━━━━━ 22s 126ms/step - accuracy: 0.9110 - loss: 0.2472 - val_accuracy: 0.9097 - val_loss: 0.2801
Epoch 3/10
170/170 ━━━━━━━━━━━━━━━━━━━━ 22s 126ms/step - accuracy: 0.9139 - loss: 0.2318 - val_accuracy: 0.9083 - val_loss: 0.2752
Epoch 4/10
170/170 ━━━━━━━━━━━━━━━━━━━━ 22s 126ms/step - accuracy: 0.9247 - loss: 0.2124 - val_accuracy: 0.9140 - val_loss: 0.2767
Epoch 5/10
170/170 ━━━━━━━━━━━━━━━━━━━━ 22s 127ms/step - accuracy: 0.9287 - loss: 0.1953 - val_accuracy: 0.9126 - val_loss: 0.2675
Epoch 6/10
170/170 ━━━━━━━━━━━━━━━━━━━━ 22s 127ms/step - accuracy: 0.9357 - loss: 0.1787 - val_accuracy: 0.9097 - val_loss: 0.2716
Epoch 7/10
170/170 ━━━━━━━━━━━━━━━━━━━━ 22s 128ms/step - accuracy: 0.9437 - loss: 0.1691 - val_accuracy: 0.9083 - val_loss: 0.2727
Epoch 8/10
170/170 ━━━━━━━━━━━━━━━━━━━━ 22s 128ms/step - accuracy: 0.9427 - loss: 0

confusion matrix

In [30]:
import numpy as np
from sklearn.metrics import confusion_matrix

In [32]:
y_true=np.concatenate([y for x, y in val_ds], axis=0)
y_pred=model.predict(val_ds)
y_pred=y_pred.argmax(axis=1)
y_true=y_true.argmax(axis=1)

confusion_matrix(y_true, y_pred)

23/23 ━━━━━━━━━━━━━━━━━━━━ 6s 157ms/step


array([[48, 24, 36, 40, 28],
       [21, 18, 16, 40, 11],
       [36, 20, 29, 35, 25],
       [39, 27, 43, 45, 28],
       [21, 21, 19, 28, 11]])

In [40]:
from tensorflow.keras.preprocessing import image
import numpy as np

img = image.load_img("/kaggle/input/5-vehichles-for-multicategory-classification/dataset/test/motorcycle/109.png", target_size=(224,224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)

pred = model.predict(img)
class_id = np.argmax(pred)

if class_id==0:
    print('BUS')
elif class_id==1:
    print('CAR')
elif class_id==2:
    print('MOTORCYCYLE')
elif class_id==3:
    print('TRAIN')
else:
    print('TRUCK')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
MOTORCYCYLE


In [41]:
model.save('multiclass vehicle.h5')